# Install Packages & Import Libraries

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium
!pip install -q google-colab-selenium
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.0 MB/s eta 0:00:00


I use BeautifulSoup for this task to make it simpler and it could get the data I needed.

In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

# Scraping Process

In [ ]:
def auto_Scrapper_Class(html_tag,course_case,tag_class, div_class=None):

    for i in range(1,50): # according to current coursera website, there are 84 pages for all courses
        url = "https://www.coursera.org/courses?page=" +str(i)

        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        if div_class:
            elements = soup.find_all('div',  class_ = div_class)

            if (len(elements)) != 12:
                for j in range(0,12): # There are 12 courses per page
                    course_case.append(None)
                continue
            for name in elements:
                x = name.get_text()
                if x:
                    course_case.append(x)
                else:
                    course_case.append(None)

        else:
            element = soup.find_all(html_tag,  class_ = tag_class)
            if (len(element)) != 12:
                for j in range(0,12):
                    course_case.append(None)
                continue

            for name in element:
                x = name.get_text()
                if x:
                    course_case.append(x)
                else:
                    course_case.append("")

In [ ]:
course_title = []
course_organization = []
course_Certificate_type = []
course_rating = []
difficulty = []
course_review_counts = []
course_skills = []

In [ ]:
# scraping course title
auto_Scrapper_Class('h3',course_title, tag_class='cds-119 cds-CommonCard-title css-e7lgfl cds-121')

In [ ]:
#scrap the other information as per coursera's website html
auto_Scrapper_Class('p',course_organization,'cds-119 cds-ProductCard-partnerNames css-dmxkm1 cds-121')

#auto_Scrapper_Class('div',course_Certificate_type,'_jen3vs _1d8rgfy3')
auto_Scrapper_Class('p',course_rating,'cds-119 css-11uuo4b cds-121')
auto_Scrapper_Class('p',difficulty,'cds-119 cds-Typography-base css-dmxkm1 cds-121', 'cds-CommonCard-metadata')
auto_Scrapper_Class('p',course_review_counts,'cds-119 cds-Typography-base css-dmxkm1 cds-121', 'product-reviews css-pn23ng')
auto_Scrapper_Class('p',course_skills,'cds-119 cds-Typography-base css-dmxkm1 cds-121', 'cds-CommonCard-bodyContent' )

In [ ]:
data = {
    'Title': course_title,
    'Organization': course_organization,
    'Skills': course_skills,
    'Ratings':course_rating,
    'Review counts':course_review_counts,
    'Metadata': difficulty
}
df = pd.DataFrame(data)
df['Skills'] = df['Skills'].str.replace("Skills gained:", '', regex=False)
df

,Title,Organization,Skills,Ratings,Review counts,Metadata
0,None,None,"Data Storytelling, Rmarkdown, Data Literacy, ...",None,None,Beginner · Professional Certificate · 3 - 6 Mo...
1,None,None,"Threat Modeling, Network Security, Incident R...",None,None,Beginner · Professional Certificate · 3 - 6 Mo...
2,None,None,"Quality Management, Project Management Life C...",None,None,Beginner · Professional Certificate · 3 - 6 Mo...
3,None,None,"Prompt Engineering, Generative AI, Artificial...",None,None,Beginner · Specialization · 1 - 3 Months
4,None,None,"Data Storytelling, Search Engine Marketing, M...",None,None,Beginner · Professional Certificate · 3 - 6 Mo...
...,...,...,...,...,...,...
583,None,None,"Unsupervised Learning, Supervised Learning, M...",None,None,Beginner · Specialization · 1 - 3 Months
584,None,None,"Data Visualization, Prompt Engineering, Data ...",None,None,Beginner · Specialization · 1 - 4 Weeks
585,None,None,"Data Ethics, Market Opportunities, Artificial...",None,None,Beginner · Course · 1 - 4 Weeks
586,None,None,"Data Storytelling, Data Visualization, Data E...",None,None,Advanced · Professional Certificate · 3 - 6 Mo...


In [ ]:
skills_column = df['Skills']
skills_column = [str(skill)
  if skill is not None
  else '' for skill in skills_column]

# Concatenate
all_skills_text = ', '.join(skills_column)

# Split the string into a list of skills
all_skills_list = [skill.strip() for skill in all_skills_text.split(',')]

distinct_skills = list(set(all_skills_list))

distinct_skills_df = pd.DataFrame({'Distinct Skills': distinct_skills})
len(distinct_skills)

183

I decided to take only the detailed skills to make it easier for the preprocessing code.

In [ ]:
distinct_skills_df

,Distinct Skills,source
0,Git (Version Control System),coursera
1,Threat Modeling,coursera
2,Quality Management,coursera
3,Engineering Management,coursera
4,Process Optimization,coursera
...,...,...
178,Threat Management,coursera
179,Object Oriented Programming (OOP),coursera
180,Hardening,coursera
181,Data Visualization,coursera


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

def preprocess_text(text):

    if not isinstance(text, str):
        return ""

    # Handle NaN
    if pd.isna(text) or text.strip() == "":
        return ""

    # Normalization
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if not text:
        return ""

    # Tokenization
    try:
        tokens = word_tokenize(text)
    except Exception as e:
        print(f"Tokenization error for text: {text[:50]}... Error: {e}")
        return ""

    # Delete stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

# Apply preprocessing
try:
    distinct_skills_df['Processed_Skills'] = distinct_skills_df['Distinct Skills'].apply(preprocess_text)

    # Display the dataframe with the new processed column
    print("Original vs Processed Skills:")
    print(distinct_skills_df[['Distinct Skills', 'Processed_Skills']].head())


except Exception as e:
    print(f"Error processing dataframe: {e}")
    print("Make sure 'distinct_skills_df' exists and has a 'Distinct Skills' column")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Original vs Processed Skills:
                Distinct Skills            Processed_Skills
0  Git (Version Control System)  git version control system
1               Threat Modeling                threat model
2            Quality Management               qualiti manag
3        Engineering Management                 engin manag
4          Process Optimization               process optim

Total rows processed: 183
Empty processed skills: 0


In [ ]:
output_filename = 'HandsOn1_coursera.csv'
distinct_skills_df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"\nData saved to: {output_filename}")


Data saved to: HandsOn1_coursera.csv
